In [1]:
import gensim
import os
import json
import random
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd

C:\Users\rapha\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
with open('lda_danbooru.pkl', 'rb') as file:
    LDA_danbooru = pickle.load(file)

In [3]:
with open('training.pkl', 'rb') as file:
    training_data = pickle.load(file)
with open('test.pkl', 'rb') as file:
    test_data = pickle.load(file)

In [2]:
metadata = 'D:/UMSI Classes/Fall2018/670/Project/danbooru2017/metadata/'

## Processing step

In [4]:
'''
Extract tags from SFW pictures - 226k pictures, should have the picture on file.
Take only tags that have 0 category (general tags, not series, character, artist, or Danbooru meta)
'''
i = 0
tags = {}

list_of_jsons = os.listdir(metadata)

for metadata_json in list_of_jsons:
    json_path = os.path.join(metadata+metadata_json)
    print('Running on json file {}'.format(metadata_json))
    with open(json_path, 'r', encoding='utf-8') as file:
        random.seed(453432)
        for num, line in enumerate(file):
            try:
                json_set = json.loads(line)
                #check if id exists and is safe for work

                if int(json_set['id'])%1000 >= 100 or json_set['rating'] != 's':
                    continue

                img_id = json_set['id']

                img_tags = [tag['name'] for tag in json_set['tags'] if tag['category'] == '0']

                tags[img_id] = img_tags

                if i%10000 == 0:
                    print(i)

                i+= 1

            except Exception:
                print(line)

Running on json file 2017000000000000.json
0
10000
Running on json file 2017000000000001.json
20000
Running on json file 2017000000000002.json
30000
Running on json file 2017000000000003.json
40000
50000
Running on json file 2017000000000004.json
60000
Running on json file 2017000000000005.json
70000
Running on json file 2017000000000006.json
80000
90000
Running on json file 2017000000000007.json
100000
Running on json file 2017000000000008.json
110000
Running on json file 2017000000000009.json
120000
130000
Running on json file 2017000000000010.json
140000
Running on json file 2017000000000011.json
150000
Running on json file 2017000000000012.json
160000
170000
Running on json file 2017000000000013.json
180000
Running on json file 2017000000000014.json
190000
Running on json file 2017000000000015.json
200000
210000
Running on json file 2017000000000016.json
220000


In [5]:
all_tags = [taglist for taglist in tags.values()]

len(all_tags)

226484

In [6]:
from gensim import corpora
from gensim.models import LdaModel
tag_dict = corpora.Dictionary(all_tags)
tag_dict.filter_extremes(no_below=5)

bag_of_words = [tag_dict.doc2bow(taglist) for taglist in tags.values()]

In [19]:
if LDA_danbooru:
    print('Already loaded')
    pass
else:
    LDA_danbooru = LdaModel(bag_of_words, num_topics=10, id2word=tag_dict)
    with open('lda_danbooru.pkl', 'wb') as file:
        pickle.dump(LDA_danbooru, file)

Already loaded


In [20]:
training_data_ids = [x[0] for x in training_data]
training_data_tags = [tags[str(x)] for x in training_data_ids]

test = tag_dict.doc2bow(training_data_tags[0]) 

In [21]:
LDA_danbooru.get_document_topics(test)

[(0, 0.123928085),
 (1, 0.011117434),
 (2, 0.011111391),
 (3, 0.40599573),
 (4, 0.011113007),
 (5, 0.011112221),
 (6, 0.011112949),
 (7, 0.011111698),
 (8, 0.01111281),
 (9, 0.3922847)]

In [38]:
def get_doc_weights(img_id):
    final_topics = [0]*10
    img_id = str(img_id)
    taglist = tags[img_id]
    bow_taglist = tag_dict.doc2bow(taglist)
    topics = LDA_danbooru.get_document_topics(bow_taglist)
    for pair in topics:
        final_topics[pair[0]] = pair[1]
    
    return final_topics

In [41]:
training_data_LDA = [(x[0], x[1], get_doc_weights(x[0])) for x in training_data]
test_data_LDA = [(x[0], x[1], get_doc_weights(x[0])) for x in test_data]

In [42]:
with open('training_LDA.pkl', 'wb') as file:
    pickle.dump(training_data_LDA, file)
with open('test_LDA.pkl', 'wb') as file:
    pickle.dump(test_data_LDA, file)